In [24]:
import numpy as np
import tensorflow as tf
import random
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, Input
from tensorflow.keras import Model
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.regularizers import l2
from sklearn.utils import shuffle

import Dataset_pb2

In [3]:
BOARD_SIZE = 9;
CHANNELS = 2;


In [4]:
data_set = Dataset_pb2.DataSet()
f = open('../resources/dataset_mcts400_forced3_all.small.bin', "rb")
data_set.ParseFromString(f.read())
f.close()

In [49]:
test_data = []
train_data = []
for data_instance in data_set.data:
    data.append(data_instance)
    total_games = data_instance.game_id + 1
data = random.sample(data, len(data))

In [50]:
total_games

1000

In [133]:
states = [];
values = [];
policies = [];
for data_instance in data:
    board_state = np.array(data_instance.state)
    board_state = board_state.reshape(BOARD_SIZE, BOARD_SIZE, CHANNELS)
    symetric_states = [board_state]
    symetric_states = symetric_states + get_rotated(board_state)
    symetric_states = symetric_states + get_flipped(board_state)
    for state in symetric_states:
        states.append(state)
        values.append(data_instance.value)
        #TODO flip policies
        policies.append(data_instance.policy)
states = np.array(states)
values = np.array(values)
policies = np.array(policies)

In [135]:
toNCHW(states[0])

array([[[ 1.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  1.],
        [ 1.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  1.],
        [ 0.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  0.],
        [ 1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.],
        [ 1.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,  1.],
        [ 0.,  1.,  0.,  1.,  0.,  0.,  1.,  1.,  0.],
        [ 1.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  0.,  1., 10.,  1.,  1.,  1.],
        [ 1.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  1.]],

       [[ 0.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,  1.],
        [ 1.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  1.],
        [ 1.,  1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 0.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  1.],
        [ 1.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.],
        [ 1.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  0.]]])

In [120]:
toNCHW(get_flipped(states[0])[0])

array([[[ 1.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  1.],
        [ 1.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  1.],
        [ 0.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  1.],
        [ 1.,  0.,  0.,  1.,  1.,  1.,  0.,  1.,  1.],
        [ 0.,  1.,  1.,  0.,  0.,  1.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.],
        [ 1.,  1.,  1., 10.,  1.,  0.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  0.,  1.,  1.,  0.,  1.]],

       [[ 1.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,  0.],
        [ 1.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  1.],
        [ 0.,  1.,  0.,  0.,  0.,  1.,  1.,  1.,  1.],
        [ 1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.],
        [ 0.,  1.,  1.,  0.,  1.,  0.,  0.,  1.,  1.],
        [ 1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  1.]]])

In [98]:
def get_rotated(board_state):
    ret = []
    for rot in range(1, 4):
        rotated_board_state = np.rot90(board_state, k=rot)
        ret.append(rotated_board_state)
    return ret


def get_flipped(board_state):
    ret = []
    for axis in range(1, 3):
        flipped_board_state = np.flip(board_state, axis=axis).copy()
        ret.append(flipped_board_state)
    return ret

In [103]:
def toNCHW(board_state):
    return np.moveaxis(board_state, 2, 0)

In [18]:
# states_3d = states.reshape((len(states), BOARD_SIZE, BOARD_SIZE, CHANNELS))
# change channels last to channels first format
states_NCHW = np.moveaxis(states, 3, 1)

In [80]:
policies.reshape(len(states), BOARD_SIZE, BOARD_SIZE)[0]

array([[0.035 , 0.0425, 0.0375, 0.0225, 0.    , 0.    , 0.0225, 0.    ,
        0.035 ],
       [0.    , 0.    , 0.    , 0.03  , 0.    , 0.    , 0.02  , 0.    ,
        0.025 ],
       [0.0375, 0.    , 0.    , 0.0425, 0.0325, 0.03  , 0.03  , 0.    ,
        0.03  ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.025 , 0.    , 0.    ,
        0.035 ],
       [0.0425, 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    ],
       [0.035 , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.0325],
       [0.0375, 0.0275, 0.    , 0.    , 0.    , 0.    , 0.    , 0.0225,
        0.0375],
       [0.    , 0.0275, 0.0325, 0.03  , 0.    , 0.    , 0.0275, 0.025 ,
        0.    ],
       [0.    , 0.    , 0.    , 0.0375, 0.    , 0.0275, 0.025 , 0.    ,
        0.    ]])

In [52]:
states_NCHW[0]

array([[[ 0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  1.,  0.,  1.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.],
        [ 0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0., 10.,  0.,  0.,  0.],
        [ 0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]])

In [112]:
train_ds = tf.data.Dataset.from_tensor_slices((states_3d, values)).shuffle(10000).batch(32)

In [18]:
# Reserve 10,000 samples for validation
# x_val = states_3d[-10000:]
# y_val = values[-10000:]
# x_train = states_3d[:-10000]
# y_train = values[:-10000]

In [53]:
# class CNNModel(Model):
#     def __init__(self):
#         super(Model, self).__init__()
#         self.conv1 = Conv2D(32, 3, padding='same', activation='relu')
#         self.pool1 = MaxPool2D((2,2))
#         self.conv2 = Conv2D(64, 3, padding='same', activation='relu')
#         self.pool2 = MaxPool2D((2,2))
#         self.flatten = Flatten()
#         self.d1 = Dense(512, activation='relu')
#         self.dropout1 = Dropout(0.4)
#         self.d2 = Dense(128, activation='relu')
#         self.dropout2 = Dropout(0.4)
#         self.d3 = Dense(43, activation='softmax')

#     def call(self, x):
#         x = self.conv1(x)
#         x = self.pool1(x)
#         x = self.conv2(x)
#         x = self.pool2(x)
#         x = self.flatten(x)
#         x = self.d1(x)
#         x = self.dropout1(x)
#         x = self.d2(x)
#         x = self.dropout2(x)
#         x = self.d3(x)
#         return x
# model = CNNModel()
L2_VALUE = 1e-4  # coef of l2 penalty 

# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(2, 9, 9)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.Conv2D(128, (3, 3), activation='relu'))

# model.add(layers.Conv2D(32, activation="relu", kernel_size=(3, 3),
#                  input_shape=(BOARD_SIZE, BOARD_SIZE, CHANNELS),
#                  data_format="channels_last",
#                  padding='same'))
# model.add(layers.Conv2D(32, activation="relu", kernel_size=(3, 3),
#                  data_format="channels_last",
#                  padding='same'))
# model.add(layers.MaxPooling2D((2, 2), data_format="channels_last"))
# model.add(layers.Conv2D(64 * 2, activation="relu", kernel_size=(3, 3),
#                  data_format="channels_last",
#                  padding='same'))
# model.add(layers.Conv2D(64 * 2, activation="relu", kernel_size=(3, 3),
#                  data_format="channels_last",
#                  padding='same'))
# model.add(layers.MaxPooling2D((2, 2), data_format="channels_last"))
# model.add(layers.Flatten())
# model.add(layers.Dense(32, activation='relu'))
# model.add(layers.Dense(1, activation='tanh'))

in_x = network = Input((BOARD_SIZE, BOARD_SIZE, CHANNELS),)
# conv layers
network = Conv2D(filters=32, kernel_size=(3, 3), padding="same", data_format="channels_last", activation="relu", kernel_regularizer=l2(L2_VALUE))(network)
network = Conv2D(filters=64, kernel_size=(3, 3), padding="same", data_format="channels_last", activation="relu", kernel_regularizer=l2(L2_VALUE))(network)
network = Conv2D(filters=128, kernel_size=(3, 3), padding="same", data_format="channels_last", activation="relu", kernel_regularizer=l2(L2_VALUE))(network)
# action policy layers
policy_net = Conv2D(filters=4, kernel_size=(1, 1), data_format="channels_last", activation="relu", kernel_regularizer=l2(L2_VALUE))(network)
policy_net = Flatten()(policy_net)
policy_net = Dense(BOARD_SIZE*BOARD_SIZE, activation="softmax", kernel_regularizer=l2(L2_VALUE), name="policy_output")(policy_net)
# state value layers
value_net = Conv2D(filters=2, kernel_size=(1, 1), data_format="channels_last", activation="relu", kernel_regularizer=l2(L2_VALUE))(network)
value_net = Flatten()(value_net)
value_net = Dense(64, kernel_regularizer=l2(L2_VALUE))(value_net)
value_net = Dense(1, activation="tanh", kernel_regularizer=l2(L2_VALUE), name="value_output")(value_net)

model = Model(in_x, [policy_net, value_net])

# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [56]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='model_mcts400_forced3_all_{epoch}.h5',
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]

In [58]:
model.compile(optimizer='adam',
                  loss = {
                      "policy_output": "categorical_crossentropy",
                      "value_output": "mean_squared_error",
                  },
                  loss_weights = {"policy_output": 1.0, "value_output": 1.0}
#                 metrics=['accuracy','mae']
             )
# validation_data=(test_images, test_labels)

# Train the model by slicing the data into "batches"
# of size "batch_size", and repeatedly iterating over
# the entire dataset for a given number of "epochs"
print('# Fit model on training data')
history = model.fit(states, [policies, values],
                    batch_size=64,
                    epochs=3,
                    callbacks=callbacks,
                    validation_split=0.2)

# Fit model on training data
Train on 75083 samples, validate on 18771 samples
Epoch 1/3
75072/75083 [============================>.] - ETA: 0s - loss: 4.1093 - policy_output_loss: 3.5894 - value_output_loss: 0.4988
Epoch 00001: val_loss improved from inf to 4.04460, saving model to model_mcts400_forced3_all_1.h5
75083/75083 [==============================] - 66s 885us/sample - loss: 4.1093 - policy_output_loss: 3.5895 - value_output_loss: 0.4988 - val_loss: 4.0446 - val_policy_output_loss: 3.5332 - val_value_output_loss: 0.4942
Epoch 2/3
75072/75083 [============================>.] - ETA: 0s - loss: 3.9207 - policy_output_loss: 3.5413 - value_output_loss: 0.3575
Epoch 00002: val_loss improved from 4.04460 to 3.67340, saving model to model_mcts400_forced3_all_2.h5
75083/75083 [==============================] - 71s 943us/sample - loss: 3.9207 - policy_output_loss: 3.5413 - value_output_loss: 0.3573 - val_loss: 3.6734 - val_policy_output_loss: 3.5228 - val_value_output_loss: 0.1189
Epoch

In [59]:
# The returned "history" object holds a record
# of the loss values and metric values during training
print('\nhistory dict:', history.history)

# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(x_val, y_val, batch_size=128)
print('test loss, test acc:', results)


history dict: {'loss': [4.109343210134245, 3.9207057372957506, 3.6156365291696613], 'policy_output_loss': [3.5895114, 3.5412586, 3.52299], 'value_output_loss': [0.4988098, 0.35730165, 0.05523728], 'val_loss': [4.0445963812334185, 3.673395772828995, 3.5647402849177574], 'val_policy_output_loss': [3.5331588, 3.522752, 3.5045166], 'val_value_output_loss': [0.49422672, 0.11885377, 0.020093251]}

# Evaluate on test data


NameError: name 'x_val' is not defined

In [64]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print('\n# Generate predictions for 3 samples')
predictions = model.predict(states[:1])
print('predictions shape:', predictions[0].shape)


# Generate predictions for 3 samples
predictions shape: (1, 81)


In [65]:
model.save('model_mcts400_forced3_all.h5')